In [1]:
import pickle
import numpy as np
import pandas as pd
from keras import Sequential, backend, regularizers
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.backend
import os

Using TensorFlow backend.


In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# Load tweet data

In [3]:
# Functions for importing & cleaning relevant tweets
def lower(s):
    return s.lower()

def tweet_imports(filename):
    imp = pd.read_pickle(filename)
    imp = imp.drop_duplicates()
    imp['tweet_clean'] = imp['tweet'].str.replace('http\S+|www.\S+|pic.twitter.com\S+', '', case=False)
    imp['tweet_clean'] =imp['tweet_clean'].replace('[^A-Za-z0-9 ]+','',regex=True)
    imp['tweet_clean'] = imp['tweet_clean'].apply(lower)#map(lambda x: x.lower(), imp['tweet_clean'])
    imp['date'] = pd.to_datetime(imp['date'])
    return imp

In [8]:
cleaned_affirm_tweets = tweet_imports('all_affirm_tweets.pkl')
cleaned_deny_tweets = tweet_imports('all_deny_tweets.pkl')

In [5]:
cleaned_affirm_tweets.head()

,id,date,time,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,username,search_term,label,tweet_clean
0,1173949629337133057,2019-09-17,06:19:37,Well...lots of protests at pipeline sites. Thi...,"['chrisraab3', 'yasminrjhenry']",1,0,0,[],abarnardnyt,sea,1,welllots of protests at pipeline sites this on...
1,1172850391765135360,2019-09-14,05:31:38,"Think sea worms or sea bugs, not sea monsters ...",['publicoutcrysy'],1,0,0,[],abarnardnyt,sea,1,think sea worms or sea bugs not sea monsters ...
2,1172631555577135109,2019-09-13,15:02:03,The water in NYC is so much cleaner these days...,[],3,0,10,[],abarnardnyt,sea,1,the water in nyc is so much cleaner these days...
3,1167944666274246656,2019-08-31,16:38:02,Working for Les Gelb was also fun. He was in h...,[],1,1,8,[],abarnardnyt,sea,1,working for les gelb was also fun he was in hi...
4,1167475090411413504,2019-08-30,09:32:06,Some New York Dems seem irked by rumored choic...,[],0,1,0,['#climate'],abarnardnyt,sea,1,some new york dems seem irked by rumored choic...


In [9]:
len(cleaned_affirm_tweets)

230802

In [10]:
len(cleaned_deny_tweets)

290084

In [14]:
import csv
with open('generated.tsv','w') as f:
    csvwriter = csv.writer(f,delimiter='\t')
    for ix in range(100):#len(cleaned_deny_tweets)):
        df_row = cleaned_deny_tweets.iloc[ix]
        csvwriter.writerow([df_row['tweet_clean'],float(df_row['likes_count']),
                                  float(df_row['retweets_count']),float(df_row['replies_count']),
                            df_row['username']])
    for ix in range(100):#len(cleaned_deny_tweets)):
        df_row = cleaned_deny_tweets.iloc[-ix]
        csvwriter.writerow([df_row['tweet_clean'],float(df_row['likes_count']),
                                  float(df_row['retweets_count']),float(df_row['replies_count']),
                            df_row['username']])